# HW2 - Coffee Preferences Code

In [2]:
# Code to prepare the coffee data set. 
# pandas is used to load the CSVs; numpy is
# otherwise used throughout.

import numpy as np
import pandas as pd

assortments  = pd.read_csv("coffee_assortments_v2.csv", header = None)
transaction_counts = pd.read_csv("coffee_transaction_counts_v2.csv", header = None)
assortments = np.asarray( assortments)
transaction_counts = np.asarray( transaction_counts)

## Code to set up `permToA` function

In [3]:
# To make the code easier to read, we will use N to denote the total
# number of options, which is 9 + 1 = 10.
n = 9
N = n + 1 
M = assortments.shape[0]

# NB: the function permToA below returns a single column
# of the A matrix. Note that this "column" is actually
# formated as a 2D array (rows are assortments and range from 0
# to M-1, and columns are options which range from 0 to N-1).
# This is a deliberate design choice, as collapsing it into
# a single 1D vector requires a lot of tedious bookkeeping.

def permToA(perm):
    A_column = np.zeros( (M,N) )
    for i in range(M):
        for j in range(N):
            if (assortments[i,perm[j]] > 0.5):
                A_column[i,perm[j]] = 1.0
                break
    return A_column

## Skeleton code to set up `separation` function and complete constraint generation procedure

In [ ]:
# Below is skeleton code to set up your constraint generation procedure.
# Parts of the uncommented code contain ... 's, which are places 
# where you need to fill in the code. 

# =====================================
# First, set up your dual problem here:

m_dual = Model()
p = m_dual.addVars(M, N, ...) 
q = m_dual.addVar(...)

# Add constraints here:
...
# (bounds on p_{m,i}, and constraints corresponding to 
# initial six rankings.)
# You may find it helpful to maintain a list, A_list, which
# contains the 2D arrays generated by permToA. 
# You may also find it helpful to maintain a list, ranking_list,
# which contains the rankings corresponding to A_list. 

# Add objective function here:
...

m_dual.update()
m_dual.optimize()

# =================================================
# Set up your separation subproblem in Gurobi here: 

m_sub = Model()

A = m_sub.addVars(M,N, ...)
z = m_sub.addVars(N,N, ...)

m_sub.update()

# =====================================
# Set up your separation function here:

def separation(p_val, q_val):
    m_sub.objective(..., GRB.MAXIMIZE)
    
    m_sub.update()
    m_sub.optimize()
    
    A_val = np.asarray( [[A[m,i].x for i in range(N)] for m in range(M)] )
    z_val = np.asarray( [[z[i,j].x for j in range(N)] for i in range(N)] )
    
    # Use z_val to determine the ranking:
    ranking = ... 
    
    return sub_obj, A_val, ranking


# =======================================================
# Test your separation function below:

np.random.seed(10)
p_val = np.random.randint(-1,1,size = (M,N))
q_val = 0.0

sub_objval, single_A, single_perm = separation(p_val, q_val)

print(sub_objval) # Should display -14.0
print(single_perm) # Should display [8 5 0 6 9 7 4 3 2 1]

# =======================================================
# Finally, put your constraint generation procedure here.

m_dual.Params.OutputFlag = 0
m_dual.update()
m_dual.optimize()

dual_obj = m_dual.objval
CG_iter = 0
print("Iteration ", CG_iter, ": ", dual_obj)

p_val = np.asarray( [ [p[i,j].x for j in range(N)] for i in range(M)] )
q_val = q.x

# Solve separation problem for the first time:
sub_objval, single_A, single_ranking = ... 

# Create the flag isOptimal to decide whether the solution is optimal
# or not (does it satisfy all the dual constraints or not):
isOptimal = ...

while (not isOptimal):
    # Add the column of single_A to A_list:
    ...
    
    # Add the ranking of single_ranking to ranking_list:
    ...
    
    # Add constraint corresponding to this single_A / single_ranking
    # to dual:
    m_dual.addConstr(...)
    
    # Update and re-solve the dual:
    m_dual.update()
    m_dual.optimize()
    
    # Retrieve values of dual variables p and q, and dual objective
    # value: 
    dual_obj = ...
    p_val = ... 
    q_val = ...
    
    # Print information about this iteration:
    CG_iter += 1 
    print()
    print("Iteration ", CG_iter, ": ", dual_obj)
    
    # Solve separation problem, and update the isOptimal flag.
    sub_objval, single_A, single_perm = ...
    isOptimal = ... 
    print("sub_objval = ", sub_objval)
    print("isOptimal = ", isOptimal)